# Load word2idx

In [4]:
import numpy as np
import os

filepath = os.path.join(os.path.dirname(os.getcwd()), 'output', '20200119-21-54-36', 'vocab.npz')
vocab = np.load(filepath)
print('Loaded: %s' % filepath)
word2idx = vocab['word2idx'].item()

Loaded: /home/dmlab/jihye/GIT/charNLM-kor/output/20200119-21-54-36/vocab.npz


# Load embedding

In [6]:
filepath = os.path.join(os.path.dirname(os.getcwd()), 'output', '20200119-21-54-36', 'model_weights.h5')
with open(filepath, 'rb') as f:
    embedding = pickle.load(f)
print('Loaded: %s' % filepath)

UnpicklingError: invalid load key, 'H'.

In [1]:
import sys
import os 
base_dir = os.path.dirname(os.getcwd())
output_base_dir = os.path.join(base_dir, 'output')
sys.path.append(os.path.join(base_dir, 'source'))

In [8]:
import tensorflow as tf
from tensorflow import keras
import numpy as np
# from config import parameters
import os
from six.moves import cPickle as pickle
from LSTMCNN import LSTMCNN

output_time_dir = '20200119-21-54-36'

vocab_npz_filepath = os.path.join(output_base_dir, output_time_dir, 'vocab.npz')
param_pkl_filepath = os.path.join(output_base_dir, output_time_dir, 'model_param.pkl')
model_weights_h5_filepath = os.path.join(output_base_dir, output_time_dir, 'epoch25_119.71.h5')

def vocab_unpack(vocab):
    return vocab['idx2word'], vocab['word2idx'], vocab['idx2char'], vocab['char2idx']

def word_vector(emb_layer, word_idx):
    weights = emb_layer.weights[0]
    return weights.value()[:, word_idx]

In [10]:
vocab = np.load(vocab_npz_filepath)
print('Loaded: %s' % filepath)
word2idx = vocab['word2idx'].item()

Loaded: /home/dmlab/jihye/GIT/charNLM-kor/output/20200119-21-54-36/model_weights.h5


In [17]:
n_chars = 100
char_vocab_size = min(n_chars, len(idx2char))
char_vec_size = 15
feature_maps = [50,100,150,200,200,200,200]
kernels = [1,2,3,4,5,6,7]
batch_size = 20
seq_length = 35
max_word_l = 100
batch_norm = 0
highway_layers = 2
num_layers = 2
rnn_size = 650
dropout = 0.5
n_words = len(word2idx)
word_vocab_size = min(n_words, len(idx2word))
learning_rate = 1
max_grad_norm = 5

In [18]:
model = LSTMCNN(char_vocab_size, char_vec_size, feature_maps, kernels, batch_size, seq_length, max_word_l, batch_norm, highway_layers, num_layers, rnn_size, dropout, word_vocab_size, learning_rate, max_grad_norm)
model.load_weights(model_weights_h5_filepath)

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
chars (InputLayer)              (20, 35, 100)        0                                            
__________________________________________________________________________________________________
chars_embedding (Embedding)     (20, 35, 100, 15)    825         chars[0][0]                      
__________________________________________________________________________________________________
conv2d_8 (Conv2D)               (20, 35, 100, 50)    800         chars_embedding[0][0]            
__________________________________________________________________________________________________
conv2d_9 (Conv2D)               (20, 35, 99, 100)    3100        chars_embedding[0][0]            
__________________________________________________________________________________________________
conv2d_10 

In [19]:
TARGET_WORD = '활용'
word_vector_target = model.layers[-1].weights[0].value()[:, word2idx[TARGET_WORD]]

with tf.Session() as sess:
    init = tf.global_variables_initializer()
    sess.run(init)
    word_vector_target = sess.run(word_vector_target)
    print('word_vector.shape: ', word_vector_target.shape)
    print(word_vector_target)

word_vector.shape:  (650,)
[-4.27598087e-03  1.44974422e-03 -1.07642161e-02 -6.22084364e-04
 -9.06876847e-03 -3.47432680e-04  1.11060366e-02  1.97811518e-03
 -3.16059496e-03 -6.82480680e-03  5.42516448e-03 -2.75700353e-03
  8.32411461e-03  1.49511080e-03  6.01585209e-03  7.72050209e-03
  5.60015813e-03 -1.02651222e-02  2.25885306e-03  5.58967330e-03
 -8.95429682e-03 -3.51378787e-03 -9.28675290e-03  2.50346493e-03
  8.33123177e-03  1.08734649e-02 -8.79929401e-03 -6.13281410e-03
 -5.68647683e-03  3.30726616e-03  4.07281332e-03 -1.20424759e-03
  5.65060228e-03 -3.00095510e-03  8.18910077e-04  4.66230698e-03
 -7.71160889e-03  3.22903041e-03  1.09900068e-02 -1.36132259e-03
  2.95988936e-03  2.99678743e-03 -7.37839006e-03 -6.14701910e-03
  9.87651758e-04  7.38847442e-03 -8.65919888e-03  5.86336479e-04
  1.87048502e-03 -4.70602186e-03 -5.79824857e-03  1.53048988e-03
 -8.92487913e-03  8.22200999e-03 -8.05776473e-03 -5.08226501e-03
 -4.07465268e-03  9.60774533e-03  9.76888090e-03  7.14838877e-0

In [33]:
embedding_tensor = model.layers[-1].weights[0].value()
type(embedding_tensor)

tensorflow.python.framework.ops.Tensor

In [37]:
#embedding = embedding_tensor.eval()


with tf.Session() as sess:
    init = tf.global_variables_initializer()
    sess.run(init)
    embedding = sess.run(embedding_tensor)
    
type(embedding)

numpy.ndarray

In [41]:
embedding.shape

(650, 47926)

In [44]:
np.transpose(embedding).shape

(47926, 650)